In [ ]:
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def exploracion_eda(df, columnas_seleccionadas, columna_fecha):
    """
    Realiza un análisis exploratorio de datos (EDA) en columnas específicas de un DataFrame,
    incluyendo distribución, correlación y histogramas de fecha con variables numéricas.

    Args:
        df (pd.DataFrame): El DataFrame de Pandas a analizar.
        columnas_seleccionadas (list): Una lista de las 4 columnas específicas a analizar.
        columna_fecha (str): El nombre de la columna que contiene los datos de fecha.
    """

    print("🚀 Iniciando Análisis Exploratorio de Datos (EDA) 🚀\n")

    # Verificar que las columnas existan en el DataFrame
    for col in columnas_seleccionadas + [columna_fecha]:
        if col not in df.columns:
            print(f"❌ Error: La columna '{col}' no se encuentra en el DataFrame. Por favor, verifica los nombres.")
            return

    df_seleccionado = df[columnas_seleccionadas + [columna_fecha]].copy()

    # Convertir la columna de fecha a formato datetime
    try:
        df_seleccionado[columna_fecha] = pd.to_datetime(df_seleccionado[columna_fecha])
    except Exception as e:
        print(f"⚠️ Advertencia: No se pudo convertir la columna '{columna_fecha}' a datetime. Error: {e}")
        print("Continuando el análisis sin conversión de fecha para esta parte.")

    print(f"✨ Columnas seleccionadas para el análisis: {columnas_seleccionadas} y {columna_fecha}\n")

    # --- 1. Información General y Estadísticas Descriptivas ---
    print("--- 1. Información General y Estadísticas Descriptivas ---\n")
    print("📋 Información del DataFrame (primeras filas):\n")
    print(df_seleccionado.head())
    print("\n")

    print("📊 Estadísticas descriptivas de las columnas seleccionadas:\n")
    print(df_seleccionado[columnas_seleccionadas].describe().T)
    print("\n")

    print("🔎 Valores nulos por columna:\n")
    print(df_seleccionado[columnas_seleccionadas + [columna_fecha]].isnull().sum())
    print("\n" + "-"*70 + "\n")

    # --- 2. Análisis de Distribución (para columnas numéricas continuas) ---
    print("--- 2. Análisis de Distribución (Histogramas y Boxplots) ---\n")
    numeric_cols = df_seleccionado[columnas_seleccionadas].select_dtypes(include=np.number).columns.tolist()

    if not numeric_cols:
        print("ℹ️ No hay columnas numéricas continuas entre las seleccionadas para el análisis de distribución.")
    else:
        for col in numeric_cols:
            plt.figure(figsize=(12, 5))

            # Histograma
            plt.subplot(1, 2, 1)
            sns.histplot(df_seleccionado[col].dropna(), kde=True, bins=30)
            plt.title(f'Distribución de {col}')
            plt.xlabel(col)
            plt.ylabel('Frecuencia')

            # Boxplot
            plt.subplot(1, 2, 2)
            sns.boxplot(y=df_seleccionado[col].dropna())
            plt.title(f'Boxplot de {col}')
            plt.ylabel(col)

            plt.tight_layout()
            plt.show()
            print(f"📊 La distribución de '{col}' muestra una asimetría de {df_seleccionado[col].skew():.2f} y una curtosis de {df_seleccionado[col].kurtosis():.2f}.\n")
            print("-" * 70 + "\n")
    print("\n" + "-"*70 + "\n")

    # --- 3. Análisis de Correlación (entre columnas numéricas) ---
    print("--- 3. Análisis de Correlación ---\n")
    if len(numeric_cols) > 1:
        corr_matrix = df_seleccionado[numeric_cols].corr()
        print("🔗 Matriz de Correlación de Pearson:\n")
        print(corr_matrix)
        print("\n")

        plt.figure(figsize=(8, 6))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
        plt.title('Mapa de Calor de la Matriz de Correlación')
        plt.show()
        print("💡 Los valores cercanos a 1 o -1 indican una fuerte correlación positiva o negativa, respectivamente.")
    else:
        print("ℹ️ Se necesitan al menos dos columnas numéricas para calcular la correlación.")
    print("\n" + "-"*70 + "\n")

    # --- 4. Histograma de la Fecha con cada Variable Numérica Continua ---
    print(f"--- 4. Histograma de '{columna_fecha}' con cada Variable Numérica Continua ---\n")
    if not numeric_cols:
        print(f"ℹ️ No hay columnas numéricas continuas para graficar con '{columna_fecha}'.")
    elif columna_fecha not in df_seleccionado.columns or not pd.api.types.is_datetime64_any_dtype(df_seleccionado[columna_fecha]):
        print(f"⚠️ No se puede generar el histograma de la fecha con variables numéricas porque '{columna_fecha}' no es un tipo de dato datetime válido.")
    else:
        # Extraer el año y el mes para una mejor visualización si la fecha es muy granular
        df_seleccionado['año_mes'] = df_seleccionado[columna_fecha].dt.to_period('M')
        df_seleccionado = df_seleccionado.sort_values(by=columna_fecha) # Asegurar orden temporal

        for col in numeric_cols:
            plt.figure(figsize=(15, 6))
            sns.lineplot(x=df_seleccionado['año_mes'].astype(str), y=df_seleccionado[col])
            plt.title(f'{col} a lo largo del tiempo (por Año-Mes)')
            plt.xlabel('Año-Mes')
            plt.ylabel(col)
            plt.xticks(rotation=45, ha='right')
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()
            print(f"📈 Este gráfico muestra la tendencia de '{col}' a lo largo del tiempo, agrupado por año y mes.")
            print("-" * 70 + "\n")

    print("✅ Análisis Exploratorio de Datos completado.")

# --- Ejemplo de Uso ---
if __name__ == "__main__':
    # Crear un DataFrame de ejemplo para demostrar la función
    np.random.seed(42)
    data = {
        'Fecha': pd.to_datetime(pd.date_range(start='2020-01-01', periods=100, freq='D').tolist() +
                                pd.date_range(start='2020-01-01', periods=100, freq='D').tolist()), # Duplicar para simular más datos
        'Variable_A': np.random.rand(200) * 100,
        'Variable_B': np.random.randn(200) * 10 + 50,
        'Variable_C': np.random.randint(1, 100, 200),
        'Variable_D': np.random.normal(loc=100, scale=15, size=200),
        'Categoria': np.random.choice(['X', 'Y', 'Z'], 200),
        'Otra_Numerica': np.random.lognormal(mean=2, sigma=0.5, size=200)
    }
    df_ejemplo = pd.DataFrame(C:\Users\ricar\Documents\UVG-CUARTO AÑO\8vo Semestre\DATA SCIENCE)

    # Introducir algunos valores nulos para demostración
    df_ejemplo.loc[np.random.choice(df_ejemplo.index, 10), 'Variable_A'] = np.nan
    df_ejemplo.loc[np.random.choice(df_ejemplo.index, 5), 'Variable_C'] = np.nan

    # Definir las 4 columnas específicas y la columna de fecha que quieres analizar
    mis_columnas = ['Variable_A', 'Variable_B', 'Variable_C', 'Variable_D']
    mi_columna_fecha = 'Fecha'

    # Llamar a la función de análisis exploratorio
    exploracion_eda(df_ejemplo, mis_columnas, mi_columna_fecha)